In [2]:
from transformers import pipeline

import numpy as np
import pandas as pd

from google.colab import userdata
HF = userdata.get('HF_TOKEN')

In [3]:
from huggingface_hub import login
login(HF)

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("manashjyotiborah/top-10000-movies-hosted-on-tmdb")

print("Path to dataset files:", path)

100%|██████████| 16.9M/16.9M [00:01<00:00, 9.73MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/manashjyotiborah/top-10000-movies-hosted-on-tmdb/versions/2


In [5]:
keywords_df = pd.read_csv(f'{path}/keywords_dataframe.csv', index_col='id')
mov_df = pd.read_csv(f'{path}/movies_dataset.csv', index_col='id')

keywords_df.drop(['Unnamed: 0'], axis=1, inplace=True)

movies_df = mov_df.merge(keywords_df, on='id')
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

movies_df.head()

,genre_ids,original_language,overview,popularity,release_date,title,vote_average,vote_count,genres,keywords
id,,,,,,,,,,
385687,"[28, 80, 53]",en,Over many missions and against impossible odds...,4654.279,2023-05-17,Fast X,7.3,2093,"Action, Crime, Thriller","[{'id': 9663, 'name': 'sequel'}, {'id': 9748, ..."
697843,"[28, 53]",en,Tasked with extracting a family who is at the ...,2494.010,2023-06-09,Extraction 2,7.7,910,"Action, Thriller","[{'id': 3070, 'name': 'mercenary'}, {'id': 966..."
603692,"[28, 53, 80]",en,"With the price on his head ever increasing, Jo...",1920.127,2023-03-22,John Wick: Chapter 4,7.9,3344,"Action, Thriller, Crime","[{'id': 242, 'name': 'new york city'}, {'id': ..."
569094,"[28, 12, 16, 878]",en,"After reuniting with Gwen Stacy, Brooklyn’s fu...",2013.795,2023-05-31,Spider-Man: Across the Spider-Verse,8.6,1796,"Action, Adventure, Animation, Science Fiction","[{'id': 2858, 'name': 'sacrifice'}, {'id': 328..."
502356,"[16, 10751, 12, 14, 35]",en,"While working underground to fix a water main,...",1539.037,2023-04-05,The Super Mario Bros. Movie,7.8,5165,"Animation, Family, Adventure, Fantasy, Comedy","[{'id': 282, 'name': 'video game'}, {'id': 690..."


In [15]:
movies_df.dropna(axis=0, inplace=True)

In [6]:
genres_list = list(movies_df['genres'].unique())
gl = []
for g in genres_list:
    try:
        gl.extend(g.split(', '))
    except:
        continue

GENRES = list(set(gl))
GENRES

['Adventure',
 'Fantasy',
 'Music',
 'Thriller',
 'Horror',
 'Drama',
 'Action',
 'Western',
 'Animation',
 'Mystery',
 'History',
 'Romance',
 'Crime',
 'Family',
 'War',
 'Documentary',
 'Comedy',
 'TV Movie',
 'Science Fiction']

In [18]:
data = movies_df[['overview', 'genres']].copy()

In [20]:
classifier = pipeline("zero-shot-classification")
result = classifier(
    data['overview'].to_list()[:300],
    candidate_labels=GENRES,
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [22]:
import pickle

with open('result.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

In [21]:
result[0]

{'sequence': "Over many missions and against impossible odds, Dom Toretto and his family have outsmarted, out-nerved and outdriven every foe in their path. Now, they confront the most lethal opponent they've ever faced: A terrifying threat emerging from the shadows of the past who's fueled by blood revenge, and who is determined to shatter this family and destroy everything—and everyone—that Dom loves, forever.",
 'labels': ['Family',
  'Adventure',
  'Crime',
  'Drama',
  'Documentary',
  'War',
  'Mystery',
  'Western',
  'Action',
  'History',
  'Thriller',
  'Romance',
  'Horror',
  'Music',
  'TV Movie',
  'Animation',
  'Fantasy',
  'Science Fiction',
  'Comedy'],
 'scores': [0.5393787622451782,
  0.05868272855877876,
  0.03785441070795059,
  0.036856018006801605,
  0.03238692507147789,
  0.029787633568048477,
  0.0294906385242939,
  0.027093758806586266,
  0.026329142972826958,
  0.025958450511097908,
  0.0247113686054945,
  0.02446248196065426,
  0.021078281104564667,
  0.01741

In [ ]:
labels[0]

'Action, Crime, Thriller'